In [13]:
# Required Libraries
import pandas as pd
import os

In [14]:
# Paths to the data

# SINAN DataSUS CSV files path (modify to match your file path)
sinan_path = os.path.expanduser('~/Downloads/dbc2csv/source/csv/')

# Cleaned CSV files path (modify to match your file path)
cleaned_path = os.path.expanduser('~/Downloads/dbc2csv/datasets/')

In [15]:
"""
Load the Chikungunya data from the CSV files
The CSV files are named CHIKBRYY.csv, where YY is the last two digits of the year
The files are stored in the ~/Downloads/dbc2csv/source/csv/ directory
The columns in the CSV files are not consistent across all years
We want to identify the columns that are common to all files
"""

# List of last two digits of years for which we have CSV files (2018 to 2024)
start_year = 18
end_year = 24
assert start_year < end_year, "Start year must be less than end year"
years = range(start_year, end_year + 1)

# Dictionary to store the columns for each file
file_columns = {}

# Loop through each year, build the filename, and read the CSV
for year in years:
    file_name = f'{sinan_path}CHIKBR{str(year)}.csv'
    try:
        df = pd.read_csv(file_name, low_memory=False)
        # Save the set of columns for this file
        file_columns[file_name] = set(df.columns)
    except Exception as e:
        print(f"Error loading {file_name}: {e}")

# Ensure we have loaded at least one file before proceeding
assert file_columns, "No files were loaded. Please check your file paths."

# Find common columns: the intersection of columns across all files
common_columns = set.intersection(*file_columns.values())
print("\nColumns common to all files:")
print(common_columns)

# Compute the union of all columns (all columns that appear in any file)
all_columns = set.union(*file_columns.values())

# For columns that are not common, print which files have them and which don't.
print("\nColumns that are not common among all files:")
for col in all_columns - common_columns:
    # Extract base name (e.g., CHIKBR21) from each file path
    files_with = [os.path.splitext(os.path.basename(fname))[0] 
                    for fname, cols in file_columns.items() if col in cols]
    files_without = [os.path.splitext(os.path.basename(fname))[0] 
                        for fname, cols in file_columns.items() if col not in cols]
    print(f"Column '{col}' is present in files: {files_with} and missing in files: {files_without}")



Columns common to all files:
{'CS_ESCOL_N', 'LACO_N', 'ID_AGRAVO', 'GRAV_ENCH', 'GRAV_SANG', 'COMUNINF', 'ID_RG_RESI', 'MUNICIPIO', 'ALRM_PLAQ', 'RES_CHIKS2', 'HOSPITALIZ', 'ALRM_LIQ', 'PLAQ_MENOR', 'ID_OCUPA_N', 'CS_RACA', 'TPAUTOCTO', 'COPAISINF', 'DT_GRAV', 'SG_UF', 'LEUCOPENIA', 'GRAV_HEMAT', 'PETEQUIA_N', 'SG_UF_NOT', 'ARTRALGIA', 'MANI_HEMOR', 'RES_CHIKS1', 'DOR_RETRO', 'GRAV_EXTRE', 'ALRM_SANG', 'EPISTAXE', 'ALRM_VOM', 'IMUNOH_N', 'Unnamed: 0', 'RENAL', 'CRITERIO', 'DT_SIN_PRI', 'HEPATOPAT', 'ACIDO_PEPT', 'EVIDENCIA', 'DOR_COSTAS', 'PLASMATICO', 'CON_FHD', 'DT_PCR', 'HEMATURA', 'HIPERTENSA', 'COUFINF', 'NU_ANO', 'CONJUNTVIT', 'GRAV_CONSC', 'GRAV_MIOC', 'GRAV_INSUF', 'RESUL_SORO', 'ALRM_HEPAT', 'EVOLUCAO', 'DT_NS1', 'DOENCA_TRA', 'GENGIVO', 'ALRM_HIPOT', 'RESUL_PCR_', 'ALRM_ABDOM', 'CLASSI_FIN', 'GRAV_TAQUI', 'EXANTEMA', 'GRAV_CONV', 'DT_VIRAL', 'AUTO_IMUNE', 'SOROTIPO', 'MIALGIA', 'FEBRE', 'DT_CHIK_S1', 'CLINC_CHIK', 'RESUL_NS1', 'ID_MUNICIP', 'NU_IDADE_N', 'RESUL_VI_N', 'ID_UN

In [16]:
"""
Remove columns that are not common to all files
Create a new DataFrame with only the common columns
Concatenate all the DataFrames without the last {test_years} years into a single DataFrame called X_train
Concatenate the DataFrames from the last {test_years} years into a single DataFrame called X_test
"""

# Number of years to use for testing
test_years = 2
assert common_columns, "No common columns found. Please check your file paths."
assert len(years) > test_years, "At least {test_years + 1}  years of data are required."

# Load the train and test data for each year, keeping only the common columns
X_train = []
X_test = []

for year in years:
    file_name = f'{sinan_path}CHIKBR{str(year)}.csv'
    try:
        df = pd.read_csv(file_name, usecols=common_columns, low_memory=False)
        if year < years[-test_years]:
            X_train.append(df)
        else:
            X_test.append(df)
    except Exception as e:
        print(f"Error loading {file_name}: {e}")

# Concatenate all the DataFrames into a single train and test DataFrame
X_train = pd.concat(X_train, ignore_index=True)
X_test = pd.concat(X_test, ignore_index=True)

# Display the shape of the concatenated DataFrame
print("\nShape of the concatenated X_train DataFrame: ", X_train.shape)
print("Shape of the concatenated X_test DataFrame: ", X_test.shape)



Shape of the concatenated X_train DataFrame:  (802287, 117)
Shape of the concatenated X_test DataFrame:  (650214, 117)


In [17]:
"""
Filter the data to include only the rows where the pacient was hospitalized ("HOSPITALIZ" column is equal to 1 or to 1.0)
"""

print("\nUnique values in the 'HOSPITALIZ' column:")
print(X_train['HOSPITALIZ'].value_counts())

# Filter the data to include only the rows where the patient was hospitalized
X_train = X_train[(X_train['HOSPITALIZ'] == 1) | (X_train['HOSPITALIZ'] == 1.0)]
X_test = X_test[(X_test['HOSPITALIZ'] == 1) | (X_test['HOSPITALIZ'] == 1.0)]

# Display the shape of the filtered DataFrame
print("\nShape of the filtered X_train DataFrame: ", X_train.shape)
print("Shape of the filtered X_test DataFrame: ", X_test.shape)


Unique values in the 'HOSPITALIZ' column:
HOSPITALIZ
2.0    393418
2       73033
1.0     21844
9.0     15658
1        3629
9        2885
           1
Ø           1
J           1
ï           1
Name: count, dtype: int64

Shape of the filtered X_train DataFrame:  (21844, 117)
Shape of the filtered X_test DataFrame:  (18939, 117)


In [18]:
"""
Extract the target variable from the data (EVOLUCAO column)

1- cura
2- óbito pelo
agravo
3- óbito por outras
causas
4- óbito em
investigação
9- ignorado

Remove rows where the target variable is Nan, 3 (death by other causes), 4 (under investigation) or 9 (ignored)
Only keep rows where the target variable is 1 (cure) or 2 (death by the disease)
"""

##############################################################
# Should we drop rows where the target variable is 3 or 4?
# Dropping them results in a very unbalanced dataset
##############################################################

# Print the value counts for the 'EVOLUCAO' column in X_train
print("\nValues for 'EVOLUCAO' in X_train:")
print(X_train["EVOLUCAO"].value_counts(dropna=False))

# Print the value counts for the 'EVOLUCAO' column in X_test
print("\nValues for 'EVOLUCAO' in X_test:")
print(X_test["EVOLUCAO"].value_counts(dropna=False))



Values for 'EVOLUCAO' in X_train:
EVOLUCAO
1.0    15203
NaN     3865
9.0     1587
3.0      858
2.0      251
4.0       80
Name: count, dtype: int64

Values for 'EVOLUCAO' in X_test:
EVOLUCAO
1.0    13453
NaN     2780
9.0     1540
3.0      761
2.0      296
4.0      109
Name: count, dtype: int64


In [19]:
# Filter the data to include only the rows where the target variable is 1 or 2
X_train = X_train[(X_train['EVOLUCAO'] == 1) | (X_train['EVOLUCAO'] == 1.0) | (X_train['EVOLUCAO'] == 2) | (X_train['EVOLUCAO'] == 2.0)]
X_test = X_test[(X_test['EVOLUCAO'] == 1) | (X_test['EVOLUCAO'] == 1.0) | (X_test['EVOLUCAO'] == 2) | (X_test['EVOLUCAO'] == 2.0)]

# Remove the 'EVOLUCAO' column from X_train and save it in y_train
y_train = X_train.pop("EVOLUCAO")

# Remove the 'EVOLUCAO' column from X_test and save it in y_test
y_test = X_test.pop("EVOLUCAO")

# Change the target variable to 0 for cure and 1 for death
y_train = y_train.map({1: 0, 2: 1})
y_test = y_test.map({1: 0, 2: 1})

# Print the value counts for the 'EVOLUCAO' column in y_train
print("\nValues for 'EVOLUCAO' in y_train:")
print(y_train.value_counts(dropna=True))

# Optionally, also print the value counts for y_test
print("\nValues for 'EVOLUCAO' in y_test:")
print(y_test.value_counts(dropna=True))


Values for 'EVOLUCAO' in y_train:
EVOLUCAO
0    15203
1      251
Name: count, dtype: int64

Values for 'EVOLUCAO' in y_test:
EVOLUCAO
0    13453
1      296
Name: count, dtype: int64


In [20]:
"""
Identify and remove columns with more than missing_values_threshold missing values in X_train and X_test
"""

missing_values_threshold = 0.80
assert 0.0 <= missing_values_threshold <= 1.0, "missing_values_threshold must be between 0 and 1"

# Compute the percentage of missing values in each column of X_train
missing_values_train = X_train.isnull().mean()
missing_values_test = X_test.isnull().mean()
missing_values_mean = (missing_values_train + missing_values_test) / 2

# Print the percentage of missing values in each column of X_train
print("\nPercentage of missing values in X_train:")
print(missing_values_train)

# Print the percentage of missing values in each column of X_test
print("\nPercentage of missing values in X_test:")
print(missing_values_test)



Percentage of missing values in X_train:
Unnamed: 0    0.000000
TP_NOT        0.000000
ID_AGRAVO     0.000000
DT_NOTIFIC    0.000000
SEM_NOT       0.000000
                ...   
PLAQ_MENOR    1.000000
CON_FHD       1.000000
COMPLICA      1.000000
TP_SISTEMA    0.016954
NDUPLIC_N     0.999806
Length: 116, dtype: float64

Percentage of missing values in X_test:
Unnamed: 0    0.000000
TP_NOT        0.000000
ID_AGRAVO     0.000000
DT_NOTIFIC    0.000000
SEM_NOT       0.000000
                ...   
PLAQ_MENOR    1.000000
CON_FHD       1.000000
COMPLICA      1.000000
TP_SISTEMA    0.020874
NDUPLIC_N     0.999273
Length: 116, dtype: float64


In [21]:
cols_to_drop = missing_values_train[missing_values_mean > missing_values_threshold].index

X_train = X_train.drop(columns=cols_to_drop)
X_test = X_test.drop(columns=cols_to_drop)
print(f"\nDropped columns (more than {missing_values_threshold * 100}% missing):")
print(list(cols_to_drop))


Dropped columns (more than 80.0% missing):
['DT_CHIK_S2', 'DT_PRNT', 'DT_SORO', 'RESUL_SORO', 'DT_NS1', 'RESUL_NS1', 'DT_VIRAL', 'DT_PCR', 'SOROTIPO', 'HISTOPA_N', 'IMUNOH_N', 'DOENCA_TRA', 'DT_OBITO', 'ALRM_HIPOT', 'ALRM_PLAQ', 'ALRM_VOM', 'ALRM_SANG', 'ALRM_HEMAT', 'ALRM_ABDOM', 'ALRM_LETAR', 'ALRM_HEPAT', 'ALRM_LIQ', 'DT_ALRM', 'GRAV_PULSO', 'GRAV_CONV', 'GRAV_ENCH', 'GRAV_INSUF', 'GRAV_TAQUI', 'GRAV_EXTRE', 'GRAV_HIPOT', 'GRAV_HEMAT', 'GRAV_MELEN', 'GRAV_METRO', 'GRAV_SANG', 'GRAV_AST', 'GRAV_MIOC', 'GRAV_CONSC', 'GRAV_ORGAO', 'DT_GRAV', 'MANI_HEMOR', 'EPISTAXE', 'GENGIVO', 'METRO', 'PETEQUIAS', 'HEMATURA', 'SANGRAM', 'LACO_N', 'PLASMATICO', 'EVIDENCIA', 'PLAQ_MENOR', 'CON_FHD', 'COMPLICA', 'NDUPLIC_N']


In [22]:
"""
Save the filtered data to a new CSV file in the cleaned_path directory
"""

# Save the filtered data to a new CSV file in the cleaned_path directory
X_train.to_csv(f'{cleaned_path}X_train.csv', index=False)
y_train.to_csv(f'{cleaned_path}y_train.csv', index=False)

X_test.to_csv(f'{cleaned_path}X_test.csv', index=False)
y_test.to_csv(f'{cleaned_path}y_test.csv', index=False)